    "# TimeCopilot Weekend Exploration
",
    "
",
    "This notebook walks through TimeCopilot's capabilities step by step. You'll see how it automatically picks the best forecasting model and explains its reasoning in plain English.
",
    "
",
    "## What You'll Learn:
",
    "- How AI selects the best forecasting model from multiple options
",
    "- How to ask natural language questions about time series data
",
    "- How to interpret forecast confidence and reliability
",
    "- How different models handle different types of data patterns
",
    "
",
    "---
",
    "
",
    "## Setup
",
    "
",
    "**You'll need:**
",
    "- Python 3.8 or higher
",
    "- OpenAI API key (set as environment variable)
",
    "- Internet connection (for downloading stock data)"

In [ ]:
    "# Install required packages (run this first)
",
    "!pip install timecopilot pandas matplotlib seaborn yfinance scikit-learn numpy --quiet
",
    "
",
    "import warnings
",
    "warnings.filterwarnings('ignore')
",
    "print("All packages installed successfully!")"

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from timecopilot import TimeCopilot
import yfinance as yf
import os

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

    "# Check API key
",
    "if not os.getenv("OPENAI_API_KEY"):
",
    "    print("Warning: Please set your OpenAI API key")
",
    "    print("Add this to your environment: export OPENAI_API_KEY='your-key-here'")
",
    "else:
",
    "    print("OpenAI API key detected!")
",
    "
",
    "print("Ready to explore TimeCopilot!")"

## Step 1: Get Some Real Data

Let's start by downloading Apple stock data to see TimeCopilot in action with real financial data.

In [ ]:
    "# Download Apple stock data
",
    "print("Downloading Apple stock data...")
",
    "
",
    "# Get 2 years of Apple data
",
    "end_date = datetime.now()
",
    "start_date = end_date - timedelta(days=2*365)
",
    "
",
    "apple_data = yf.download("AAPL", start=start_date, end=end_date, progress=False)
",
    "
",
    "# Format for TimeCopilot
",
    "df = pd.DataFrame({
",
    "    'unique_id': 'AAPL',
",
    "    'ds': apple_data.index,
",
    "    'y': apple_data['Close'].values
",
    "})
",
    "df['ds'] = pd.to_datetime(df['ds'])
",
    "df = df.reset_index(drop=True)
",
    "
",
    "print(f"Downloaded {len(df)} days of Apple stock data")
",
    "print(f"Price range: ${df['y'].min():.2f} - ${df['y'].max():.2f}")
",
    "
",
    "# Quick chart
",
    "plt.figure(figsize=(12, 6))
",
    "plt.plot(df['ds'], df['y'], linewidth=1.5, alpha=0.8)
",
    "plt.title('Apple Stock Price - Last 2 Years', fontsize=14, fontweight='bold')
",
    "plt.xlabel('Date')
",
    "plt.ylabel('Stock Price ($)')
",
    "plt.grid(True, alpha=0.3)
",
    "plt.xticks(rotation=45)
",
    "plt.tight_layout()
",
    "plt.show()"

    "## Step 2: Let TimeCopilot Do Its Magic
",
    "
",
    "Now we'll see TimeCopilot's **Intelligent Forecasting Engine** automatically test different models and pick the best one."

In [ ]:
    "# Set up TimeCopilot
",
    "print("Setting up TimeCopilot...")
",
    "tc = TimeCopilot(
",
    "    llm="openai:gpt-4o-mini",  # Using mini for speed
",
    "    retries=3
",
    ")
",
    "
",
    "# What we want to know
",
    "query = """
",
    "Look at this Apple stock data and forecast the next 30 days.
",
    "I'm interested in:
",
    "1. Which model works best for stock price prediction?
",
    "2. What trends and patterns do you see?
",
    "3. How reliable is this forecast for investment decisions?
",
    "4. What are the main risk factors?
",
    """"
",
    "
",
    "print("Running forecasting analysis...")
",
    "print("This takes a minute as TimeCopilot tests different models...")"

In [ ]:
# Generate the forecast
result = tc.forecast(
    df=df,
    h=30,      # Forecast 30 days ahead
    freq="D",   # Daily frequency
    query=query
)

    "print("
Forecasting completed!")
",
    "print("
RESULTS:")
",
    "print("-" * 40)
",
    "
",
    "if hasattr(result, 'output') and result.output:
",
    "    print(f"Best Model: {result.output.selected_model}")
",
    "    print(f"
Forecast Analysis:")
",
    "    print(f"{result.output.forecast_summary}")
",
    "    
",
    "    print(f"
Technical Details:")
",
    "    print(f"{result.output.technical_details}")
",
    "else:
",
    "    print("No detailed output available")"

    "## Step 3: Ask Questions About the Forecast
",
    "
",
    "This is where TimeCopilot really shines. You can ask natural language questions and get intelligent, data-driven answers."

In [ ]:
# Let's ask some intelligent questions about our forecast
questions = [
    "What are the biggest risks in this Apple stock forecast?",
    "Should I buy, hold, or sell based on this forecast?",
    "How does Apple's recent performance compare to its historical patterns?",
    "What external factors could make this forecast inaccurate?",
    "Which days in the next 30 days are likely to have the highest prices?"
]

    "print("AI REASONING LAYER - Interactive Q&A")
",
    "print("=" * 50)
",
    "
",
    "for i, question in enumerate(questions, 1):
",
    "    print(f"
Question {i}: {question}")
",
    "    print("-" * 60)
",
    "    
",
    "    try:
",
    "        answer = tc.query(question)
",
    "        print(f"TimeCopilot: {answer.output}")
",
    "    except Exception as e:
",
    "        print(f"Error: {str(e)}")"

## 📈 Section 4: Visualize the Forecast

Let's create professional visualizations of our forecast results.

In [ ]:
# Create forecast visualizations
if hasattr(result, 'fcst_df') and result.fcst_df is not None:
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Apple Stock Forecast Analysis', fontsize=16, fontweight='bold')
    
    fcst_df = result.fcst_df
    
    # Main forecast plot
    historical = df.tail(90)  # Last 90 days for context
    forecast = fcst_df.tail(30)  # Next 30 days
    
    axes[0].plot(historical['ds'], historical['y'], 
                label='Historical Price', linewidth=2, color='blue', alpha=0.8)
    
    if result.output and result.output.selected_model in forecast.columns:
        axes[0].plot(forecast['ds'], forecast[result.output.selected_model],
                    label='Forecast', linewidth=2, color='red', linestyle='--')
        
        # Add confidence intervals if available
        model_cols = forecast.columns.tolist()
        lo_cols = [col for col in model_cols if 'lo-' in col]
        hi_cols = [col for col in model_cols if 'hi-' in col]
        
        if lo_cols and hi_cols:
            axes[0].fill_between(forecast['ds'], 
                               forecast[lo_cols[0]], 
                               forecast[hi_cols[0]],
                               alpha=0.3, color='red', label='Confidence Interval')
    
    axes[0].set_title('30-Day Stock Price Forecast')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Model performance comparison
    if hasattr(result, 'eval_df') and result.eval_df is not None:
        eval_df = result.eval_df
        if not eval_df.empty:
            model_cols = [col for col in eval_df.columns if col not in ['metric']]
            if model_cols and len(eval_df) > 0:
                mase_data = eval_df[eval_df['metric'] == 'MASE'].iloc[0] if 'MASE' in eval_df['metric'].values else None
                
                if mase_data is not None:
                    scores = [mase_data[col] for col in model_cols if col in mase_data.index]
                    colors = ['gold' if col == result.output.selected_model else 'lightblue' 
                            for col in model_cols]
                    
                    axes[1].bar(range(len(model_cols)), scores, color=colors, alpha=0.7)
                    axes[1].set_title('Model Performance\n(MASE - Lower is Better)')
                    axes[1].set_ylabel('MASE Score')
                    axes[1].set_xticks(range(len(model_cols)))
                    axes[1].set_xticklabels(model_cols, rotation=45)
                    axes[1].grid(True, alpha=0.3)
                    
                    # Highlight the best model
                    if result.output and result.output.selected_model in model_cols:
                        best_idx = model_cols.index(result.output.selected_model)
                        axes[1].bar(best_idx, scores[best_idx], color='gold', alpha=0.9)
    
    plt.tight_layout()
    plt.show()
    
    # Print forecast summary statistics
    if result.output and result.output.selected_model in forecast.columns:
        current_price = df['y'].iloc[-1]
        forecast_avg = forecast[result.output.selected_model].mean()
        forecast_max = forecast[result.output.selected_model].max()
        forecast_min = forecast[result.output.selected_model].min()
        price_change = ((forecast_avg / current_price) - 1) * 100
        
        print("\n📊 FORECAST SUMMARY:")
        print(f"   • Current Price: ${current_price:.2f}")
        print(f"   • 30-day Average Forecast: ${forecast_avg:.2f} ({price_change:+.1f}%)")
        print(f"   • Forecast Range: ${forecast_min:.2f} - ${forecast_max:.2f}")
        print(f"   • Volatility: {forecast[result.output.selected_model].std():.2f}")

else:
    print("⚠️ No forecast data available for visualization")

## 🔬 Section 5: Experiment with Your Own Questions

Now it's your turn! Ask TimeCopilot anything about the forecast. Try different types of questions to see how the AI reasons about the data.

In [ ]:
# Interactive question section
print("🎯 ASK TIMECOPILOT ANYTHING!")
print("=" * 35)
print("Examples you could try:")
print("  • 'What would happen if there's a market crash?'")
print("  • 'How confident are you in this forecast?'")
print("  • 'What's driving Apple's price movements?'")
print("  • 'Should I set a stop-loss order?'")
print("  • 'How does this compare to other tech stocks?'")
print()

# Example questions you can run
custom_questions = [
    "What would you tell a beginner investor about this Apple forecast?",
    "How sensitive is this forecast to market volatility?",
    "What's the probability that Apple will hit $200 in the next month?"
]

print("🤖 Running some example questions...\n")

for i, question in enumerate(custom_questions, 1):
    print(f"❓ Example {i}: {question}")
    print("-" * 50)
    
    try:
        answer = tc.query(question)
        print(f"🤖 TimeCopilot: {answer.output}")
        print()
    except Exception as e:
        print(f"❌ Error: {str(e)}\n")

In [ ]:
# Add your own custom question here!
your_question = "What are the key technical indicators I should watch for this Apple forecast?"

print(f"❓ Your Question: {your_question}")
print("-" * 50)

try:
    answer = tc.query(your_question)
    print(f"🤖 TimeCopilot: {answer.output}")
except Exception as e:
    print(f"❌ Error: {str(e)}")

# Feel free to change the question above and run this cell again!

## 🏁 Section 6: What's Next?

Congratulations! You've successfully explored TimeCopilot's core capabilities:

### ✅ What You've Learned:
1. **Intelligent Forecasting Engine**: How TimeCopilot automatically selects the best model from multiple options
2. **GenAI Reasoning Layer**: How AI can answer complex questions about forecasts in natural language
3. **Foundation Models**: How modern AI models handle time series forecasting
4. **Business Applications**: How this technology applies to real-world decisions

### 🚀 Next Steps:
1. **Try the other scripts**: Run `bitcoin_analysis.py`, `energy_demand.py`, or `ecommerce_sales.py`
2. **Use your own data**: Replace the Apple stock data with your own time series
3. **Experiment with different models**: Explore TimeCopilot's model selection process
4. **Ask better questions**: Practice crafting questions that get the best insights

### 📚 Additional Resources:
- [TimeCopilot Documentation](https://timecopilot.dev/)
- [Community Discord](https://discord.gg/7GEdHR6Pfg)
- [GitHub Repository](https://github.com/unbrewed/timecopilot)

In [ ]:
# Final summary
print("🎉 WEEKEND PROJECT COMPLETE!")
print("=" * 35)
print("✅ Successfully demonstrated TimeCopilot's capabilities:")
print("   • Intelligent model selection and ensemble forecasting")
print("   • Natural language reasoning about time series data")
print("   • Professional visualization and analysis")
print("   • Interactive Q&A with AI forecasting expert")
print()
print("🚀 You're now ready to use TimeCopilot for real forecasting projects!")
print("💡 Try it on your own data and share your results with the community!")